In [ ]:
import requests
from bs4 import BeautifulSoup
URL = 'https://www.gov.uk/service-standard-reports'
assessments_pg1 = requests.get(URL)
assessments_pg1_content = BeautifulSoup(assessments_pg1.content, 'html.parser')


In [ ]:
import re
pagination_label = assessments_pg1_content.find('span',class_ = 'gem-c-pagination__link-label').text.strip()
# do I need to add something if there is just one page? probably never happen?
final_page = re.search('.* of (.*)', pagination_label, re.IGNORECASE).group(1)


In [ ]:
URL = 'https://www.gov.uk/service-standard-reports?page='
i = 1
service_details = []
while i <= int(final_page):
    page = requests.get(URL+str(i))
    content = BeautifulSoup(page.content, 'html.parser')
    results = content.find_all('li', class_='gem-c-document-list__item')
    for result in results: 
        attribute_array = {"service":[],'attributes':{"url":[],"result":[],"stage":[],"assessed":[],'service_provider':[],'note':[]}}
        title_html = result.find('a', class_='gem-c-document-list__item-title gem-c-document-list__item-link')
        title_text = title_html.text.strip()
        attribute_array['service'] = title_text
        url = result.find('a')['href']
        attribute_array['attributes']['url'] = url
        attributes = result.find_all('li', class_='gem-c-document-list__attribute')
        for attribute in attributes:
            atr_text = attribute.text.strip()
            if re.match('Result: (.*)', atr_text) is not None:
                attribute = re.search('Result: (.*)', atr_text, re.IGNORECASE).group(1)
                attribute_array['attributes']['result'] = attribute 
            if re.match('Assessed: (.*)', atr_text) is not None:
                attribute = re.search('Assessed: (.*)', atr_text, re.IGNORECASE).group(1)
                attribute_array['attributes']['assessed'] = attribute
            if re.match('Stage: (.*)', atr_text) is not None:
                attribute = re.search('Stage: (.*)', atr_text, re.IGNORECASE).group(1)
                attribute_array['attributes']['stage'] = attribute
        service_details.append(attribute_array)
    i += 1

In [ ]:
service_results = []
for service_detail in service_details:
    service_result = {"service_details":[],'results':[]}
    extension = service_detail['attributes']['url']
    URL = 'https://www.gov.uk' + extension
    page = requests.get(URL)
    print(extension)
    content = BeautifulSoup(page.content, 'html.parser')
    results = content.find_all('p')
    note = content.find('p',class_='gem-c-lead-paragraph').text.strip()
    service_detail['attributes']['note'] = note
    service_result['service_details'] = service_detail
    for p in results:
        p_text = p.text.strip()
        if re.search('^The service.*of the standard', p_text, re.IGNORECASE):
            service_result['results'].append(p_text)
        if re.search('^The team.*of the standard', p_text, re.IGNORECASE):
            service_result['results'].append(p_text)
    service_results.append(service_result)
    
    if service_result['results'] == []:
        table_results = content.find_all('tr')
        for row in table_results:
            row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^Assessment date:.*', row_text, re.IGNORECASE):
                date = re.search('^(Assessment date:)(.*)', row_text, re.IGNORECASE).group(2)
                service_detail['attributes']['assessed'] = date
            if re.search('^Stage:.*', row_text, re.IGNORECASE):
                stage = re.search('^(Stage:)(.*)', row_text, re.IGNORECASE).group(2)
                service_detail['attributes']['stage'] = stage
            if re.search('^Result:.*', row_text, re.IGNORECASE):
                result = re.search('^(Result:)(.*)', row_text, re.IGNORECASE).group(2)
                service_detail['attributes']['result'] = result
            if re.search('^Service provider:.*', row_text, re.IGNORECASE):
                print(row_text)
                service_provider = ''
                service_provider = re.search('^(Service provider:)(.*)', row_text, re.IGNORECASE).group(2)
                service_detail['attributes']['service_provider'] = service_provider
            try: href = row.find('a')['href']
            except TypeError:
                href = []
            if re.search('criterion.*', str(href), re.IGNORECASE):
                for cell in row:
                    try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                    except:
                        cell_text = ''
                    if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                        point = cell_text
                    if re.search('.*met$', cell_text, re.IGNORECASE):
                        if re.search('Not met', cell_text, re.IGNORECASE):
                            result = 'did not meet'           
                        else:
                            result = 'met'           
                        service_result['results'].append('The service '+result+' point '+point+' of the Standard.')
        service_results.append(service_result)
    if service_result['results'] == []:
        table_results = content.find_all('tr')
        for row in table_results:
            row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
            for cell in row:
                try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                except:
                    cell_text = ''
                if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                    point = cell_text
                if re.search('.*Yes$|.*No$', cell_text, re.IGNORECASE):
                    if re.search('Not met', cell_text, re.IGNORECASE):
                        result = 'did not meet'           
                    else:
                        result = 'met'           
                    service_result['results'].append('The service '+result+' point '+point+' of the Standard.')
